In [ ]:
pip install glove-python


     |████████████████████████████████| 266kB 6.9MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700271 sha256=6747e47c6c79010a34ed85970f43d6e6d49cc9f4873abc0f36f82f9502c2de94
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [ ]:
from glove import Glove, Corpus

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
sen = pd.read_csv('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/분노.csv', encoding = 'utf-8')
sen2 = pd.read_csv('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/긍정.csv', encoding = 'utf-8')
sen3 = pd.read_csv('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/중립.csv', encoding = 'utf-8')

sen.columns = ['index','a']     # 컬럼명 정리
sen = sen.loc[sen['a'].notnull(),:]   # null 처리
only_word = list(sen['a'].str.split())  # 리스트형태로 만들기
rage = np.repeat('2',len(only_word))


sen2.columns = ['index','a']    # 컬럼명 정리
sen2 = sen2.loc[sen2['a'].notnull(),:]   # null 처리
only_word2 = list(sen2['a'].str.split())    # 리스트형태로 만들기
positive = np.repeat('1',len(only_word2))

sen3.columns = ['index','a']    # 컬럼명 정리
sen3 = sen3.loc[sen3['a'].notnull(),:]   # null 처리
only_word3 = list(sen3['a'].str.split())    # 리스트형태로 만들기
neutrality = np.repeat('0',len(only_word3))

total = only_word + only_word2 + only_word3
emotion = list(rage) + list(positive) + list(neutrality)

In [ ]:
df1 = pd.DataFrame({'text':only_word, 'tag' : rage})
df2 = pd.DataFrame({'text':only_word2, 'tag' : positive})
df3 = pd.DataFrame({'text':only_word3, 'tag' : neutrality})

train_x1, test_x1, train_y1, test_y1 = train_test_split(df1, df1, random_state = 0, train_size = 0.8) 
train_x2, test_x2, train_y2, test_y2 = train_test_split(df2, df2, random_state = 0, train_size = 0.8) 
train_x3, test_x3, train_y3, test_y3 = train_test_split(df3, df3, random_state = 0, train_size = 0.8)  

In [ ]:
#df1 = pd.concat([train_x1, train_x2, train_x3])
df2 = pd.concat([test_x1, test_x2, test_x3])
df1 = pd.concat([train_x1, train_x2, train_x3, train_x2, train_x2])

In [ ]:
corpus = Corpus() 
corpus.fit(df1['text'],window = 5)

In [ ]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.save('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/my_glove.model')
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [ ]:
glove.most_similar("무능")

[('중앙', 0.9374183777225747),
 ('여당', 0.9353846789631499),
 ('문재인', 0.9280535640151195),
 ('좌파', 0.9177077030481601)]

In [ ]:
glove.save('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/my_glove.txt')

In [ ]:
glove_model = glove.load('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/my_glove.model')

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/my_glove.model')
tmp_file = get_tmpfile("/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/test_word2vec.txt")
_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file, binary=False, encoding = 'utf-8')
# 인코딩 에러 발생

UnicodeDecodeError: ignored

In [ ]:
# ========================================
# 학습시킨 glove를 사용한 deep learning
# ========================================

In [ ]:
# glove의 단어와 좌표 저장
np.savetxt('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/glove-vector.tsv', glove.word_vectors, delimiter='\t') 
with open('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/glove-metadata.tsv', 'w', encoding='utf-8') as f: 
  for key in glove.dictionary.keys(): f.write(f"{key}\n")

In [ ]:
# 저장된 파일들 불러오기
df_v = pd.read_csv('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/glove-vector.tsv', sep = '\t',header = None)
df_w = pd.read_csv('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/glove-metadata.tsv', sep = '\t',header = None)

In [ ]:
# 불러온 파일 통합 
a1 = pd.concat([df_w,df_v], axis = 1).values

In [ ]:
# txt형태로 저장
with open('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/glove.naver.100d.txt','w', encoding = 'utf-8') as f :
  for i in a1 :
    f.write(f"{' '.join(list(i.astype('str')))}\n")

In [ ]:
# tokenizer 모듈을 사용
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(total)
vocab_size = len(tokenizer.word_index) + 1
X_encoded = tokenizer.texts_to_sequences(total)

max_len=max(len(l) for l in X_encoded)
print(max_len)

75


In [ ]:
# 사전학습 된 glove 모델 불러오기
import numpy as np
embedding_dict = dict()
f = open('/content/drive/MyDrive/2조 감성분석/머신러닝 팀/학습용데이터/glove.naver.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

23042개의 Embedding vector가 있습니다.


In [ ]:
# np.zeros를 사용해 비어있는 array 생성
embedding_matrix = np.zeros((vocab_size, 100))
np.shape(embedding_matrix)

(25385, 100)

In [ ]:
# glove 사전에서 단어의 좌표값을 해당되는 행에 삽입
for word, i in tokenizer.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

In [ ]:
embedding_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.35315549e-01, -5.00811934e-01, -1.71221316e-01, ...,
        -9.11829472e-01, -3.90236735e-01, -3.50935459e-01],
       [ 5.57623468e-02, -2.75628157e-02,  7.40528330e-02, ...,
        -2.27616116e-01, -7.55651912e-04,  4.54227537e-01],
       ...,
       [-6.60108356e-03,  6.91823987e-03,  1.21658379e-02, ...,
         3.61640309e-03,  1.48773543e-03,  7.55030429e-03],
       [-1.43256353e-03,  1.35796587e-03, -2.88509205e-03, ...,
         1.00190809e-03, -4.29516146e-03,  2.51416676e-03],
       [ 3.40033090e-03, -1.05350709e-03,  5.53350183e-05, ...,
        -2.36320146e-03,  1.11179659e-03, -4.75045340e-03]])

In [ ]:
# x data 문장별 정수인덱싱 후 패딩
text_sequence = tokenizer.texts_to_sequences(total)


from keras.preprocessing.sequence import pad_sequences

pad_text = pad_sequences(text_sequence, 
                         maxlen=500, 
                         truncating='post', 
                         padding='post')
train_x, test_x, train_y, test_y = train_test_split(pad_text, 
                                                    emotion, 
                                                    test_size=0.2, 
                                                    stratify=emotion, 
                                                    random_state=0)


In [ ]:
# 원 핫 인코딩
from sklearn.preprocessing import OneHotEncoder
labels_train = np.array(train_y).reshape(-1,1)
labels_test = np.array(test_y).reshape(-1,1)
onehotencoder = OneHotEncoder()
onehotencoder.fit(labels_train)
train_y = onehotencoder.transform(labels_train).toarray()
test_y = onehotencoder.transform(labels_test).toarray()

In [ ]:
# 사전학습 된 glove를 keras에 적용하기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=500, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(64, input_dim = 100 ,activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(train_x, train_y, epochs=10, batch_size=512)

Epoch 1/10
54/54 [==============================] - 9s 169ms/step - loss: 0.8374 - acc: 0.6164
Epoch 2/10
54/54 [==============================] - 9s 169ms/step - loss: 0.7427 - acc: 0.6555
Epoch 3/10
54/54 [==============================] - 9s 170ms/step - loss: 0.7033 - acc: 0.6704
Epoch 4/10
54/54 [==============================] - 9s 169ms/step - loss: 0.6813 - acc: 0.6752
Epoch 5/10
54/54 [==============================] - 9s 170ms/step - loss: 0.6640 - acc: 0.6868
Epoch 6/10
54/54 [==============================] - 9s 171ms/step - loss: 0.6507 - acc: 0.6928
Epoch 7/10
54/54 [==============================] - 9s 171ms/step - loss: 0.6354 - acc: 0.7050
Epoch 8/10
54/54 [==============================] - 9s 170ms/step - loss: 0.6213 - acc: 0.7135
Epoch 9/10
54/54 [==============================] - 9s 171ms/step - loss: 0.6101 - acc: 0.7219
Epoch 10/10
54/54 [==============================] - 9s 169ms/step - loss: 0.5974 - acc: 0.7299


In [ ]:
print(model.evaluate(test_x, test_y))

215/215 [==============================] - 2s 7ms/step - loss: 0.7368 - acc: 0.6524


[0.7368136048316956, 0.6523511409759521]

In [ ]:
# LSTM층 추가

75


NameError: ignored

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense

model = Sequential()
# 바로 이곳에 들어가는 Embedding Layer가 단어에 대하여 설정한 차원으로 변환
model.add(Embedding(vocab_size, 100, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(128, recurrent_dropout=0.1)))
model.add(Dropout(0.25))
model.add(Dense(64))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, epochs = 10, batch_size = 512)